## 🧪 Validador de Query

-   Toma la salida del generador de query, que incluye tanto la **sentencia SQL como el mensaje original del usuario**. Actúa como un juez para validar la coherencia de la query.
    1.  **Juez de coherencia**: Con la ayuda de un modelo de lenguaje, decide si la query SQL es **semánticamente coherente** con la petición del usuario.
        * Si el veredicto es **`INCOHERENTE`**, el validador no ejecuta la query y pasa el error de coherencia al corrector.
        * Si el veredicto es **`COHERENTE`**, el validador procede a ejecutar la query en la BBDD.
    2.  **Conexión y ejecución**: Se conecta a la base de datos PostgreSQL con el `search_path` configurado y, respecto a la salida:
        * ✅ **Success**: Pasa la sentencia SQL, el mensaje del usuario y la salida de la BBDD al siguiente agente (generador de conclusiones).
        * ❌ **Error**: Pasa el error de ejecución de la BBDD (sintaxis, tabla no existe, etc.) al corrector de query.

***

## 🛠️ Corrector de Query

-   Recibe un error generado por el validador, que puede ser de **dos tipos**:
    * Un error de **`incoherencia`** (el juez rechazó la query).
    * Un error de **`ejecución en la BBDD`** (la query falló en PostgreSQL).
-   Analiza el mensaje de error (o el veredicto del juez) y la query original, usando el contexto del mensaje del usuario para entender la intención.
-   Propone una **versión corregida y coherente** de la query.
-   Envía la nueva query de vuelta al validador para que intente ejecutarla nuevamente, reiniciando así el ciclo de validación.

### 1. Configuracion de entorno

In [3]:
# Instalar las librerías necesarias
!uv pip install langgraph

Using Python 3.11.13 environment at: C:\MASTER\TFM\ucm-tfm-grupo-4\.venv
Resolved 31 packages in 1m 47s
Prepared 6 packages in 8.32s
Installed 6 packages in 125ms
 + langgraph==0.6.6
 + langgraph-checkpoint==2.1.1
 + langgraph-prebuilt==0.6.4
 + langgraph-sdk==0.2.4
 + ormsgpack==1.10.0
 + xxhash==3.5.0


In [28]:
import os
import re
from dotenv import load_dotenv
from openai import AzureOpenAI
from langchain_community.utilities import SQLDatabase
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional, Any


# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# --- Configuración de Azure OpenAI ---
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

if not all([azure_api_key, azure_endpoint, azure_api_version, azure_deployment_name]):
    raise ValueError("Faltan una o más variables de entorno de Azure OpenAI.")

# Instancia del cliente de Azure OpenAI
client = AzureOpenAI(
    api_key=azure_api_key,
    azure_endpoint=azure_endpoint,
    api_version=azure_api_version
)

# --- Configuración de la conexión a la BBDD ---
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASS")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
schema_name = os.getenv("DB_SCHEMA")

db_uri = f"postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}?options=-csearch_path%3D{schema_name}"

try:
    db = SQLDatabase.from_uri(db_uri)
    print("Conexión a la base de datos establecida correctamente.")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")
    db = None

# --- Estado del Grafo ---
class GraphState(TypedDict):
    """Estado del grafo que se pasa entre los nodos."""
    user_message: str
    sql_query: str
    high_level_summary: str
    db_output: Optional[dict]
    error_message: Optional[str]
    retries: int

Conexión a la base de datos establecida correctamente.


### 2. Definicion de Nodos


In [56]:
MAX_RETRIES = 3

def validador_juez_y_query_node(state: GraphState) -> GraphState:
    sql_query = state.get("sql_query")
    user_message = state.get("user_message")
    high_level_summary = state.get("high_level_summary")
    retries = state.get("retries", 0) + 1
    
    if retries > MAX_RETRIES:
        print(f"\nLímite de {MAX_RETRIES} reintentos alcanzado. Finalizando.")
        return {"error_message": "limit_reached"}

    print("\n--- INICIANDO FASE DE JUEZ DE COHERENCIA ---")
    messages = [{"role": "system", "content": "Eres un juez experto en SQL y vas a recibir un mensaje de usuario, una query y un resumen en alto nivel de las tablas involucradas en la query. Responde SÓLO con 'COHERENTE' si la query es correcta para el mensaje, o con 'INCOHERENTE'."},
                {"role": "user", "content": f"Mensaje del usuario: '{user_message}'\nQuery SQL: '{sql_query}'\nResumen de alto nivel: '{high_level_summary}'"}]
    
    try:
        response = client.chat.completions.create(messages=messages, model=azure_deployment_name, temperature=0)
        veredicto = response.choices[0].message.content.strip().upper()
        print(f"Veredicto del juez: {veredicto}")
        
        if "INCOHERENTE" in veredicto:
            print("\n[X] La query es incoherente. No se ejecutará en la BBDD.")
            return {"error_message": "error_coherencia", "retries": retries}
    except Exception as e:
        print(f"Error al llamar al juez: {e}")
        return {"error_message": "error_juez", "retries": retries}
        
    print("\n--- PASANDO A FASE DE EJECUCIÓN EN POSTGRESQL ---")
    if not db:
        return {"error_message": "error_db", "retries": retries}

    try:
        match = re.search(r"FROM\s+(\w+)\.?(\w+)\s*", sql_query, re.IGNORECASE)
        table_name = match.group(2) if match and match.group(2) else (match.group(1) if match else None)
        
        table_info = db.get_table_info([table_name]) if table_name else None
        result_string = db.run(sql_query)
        
        db_output = {"schema": table_info, "data": result_string}
        return {"db_output": db_output, "error_message": None, "retries": retries}

    except Exception as e:
        error_message = str(e)
        print(f"\n[X] Error de PostgreSQL detectado: {error_message}")
        return {"error_message": "error_db", "retries": retries}

def corrector_de_query_node(state: GraphState) -> GraphState:
    original_query = state.get("sql_query")
    user_message = state.get("user_message")
    high_level_summary = state.get("high_level_summary")
    error_type = state.get("error_message")

    if error_type == "error_coherencia":
        prompt_content = f"La siguiente query SQL es incoherente con el mensaje del usuario, corrígela para que lo sea. Es necesario que pongas el nombre del esquema cuando referencias una tabla.. Mensaje: '{user_message}' Query: '{original_query}' Resumen tablas involucradas{high_level_summary}"
    elif error_type == "error_db":
        prompt_content = f"La siguiente query SQL ha fallado. La query era: '{original_query}'. La petición original: '{user_message}'.  Resumen tablas involucradas '{high_level_summary}'. Es necesario que pongas el nombre del esquema cuando referencias una tabla. Corrige la query."
    else:
        print("Error no reconocido. Saliendo del corrector.")
        return {"sql_query": original_query}

    messages = [{"role": "system", "content": "Eres un experto en bases de datos. Devuelve SÓLO la query corregida, sin usar MARKDOWN, sin explicaciones ni texto adicional."},
                {"role": "user", "content": prompt_content}]
    
    print(f"\n> Mandando error a Azure OpenAI para corrección...")
    try:
        response = client.chat.completions.create(messages=messages, model=azure_deployment_name, temperature=0, max_tokens=2048)
        query_corregida = response.choices[0].message.content.strip()
        query_corregida = query_corregida.replace("```sql", "").replace("```", "").rstrip(";")
        query_corregida = query_corregida.strip()
        
        print(f"  > Query corregida recibida: {query_corregida[:70]}...")
        return {"sql_query": query_corregida}
    except Exception as e:
        print(f"Error al llamar a Azure OpenAI: {e}")
        return {"sql_query": original_query}

def siguiente_tool_node(state: GraphState):
    print("\n--- LLAMADA A LA SIGUIENTE TOOL ---")
    db_output = state.get('db_output', {})
    data = db_output.get('data')
    print("La siguiente tool ha recibido los datos:")
    print(f"Mensaje del usuario: {state.get('user_message')}")
    print(f"Query SQL: {state.get('sql_query')}")
    print(f"Salida de la BBDD: {data}")
    return {}

### 3. Construcción de los Subgrafos y el Grafo Principal
Esta es la sección clave donde redefinimos la arquitectura.

#### A. Subgrafo del Validador
Este grafo se encarga de la lógica de juez y ejecución. Si la query es exitosa, termina en END_SUCCESS. Si hay un error, termina en END_ERROR.

In [57]:
validator_subgraph = StateGraph(GraphState)
validator_subgraph.add_node("validate", validador_juez_y_query_node)

validator_subgraph.add_conditional_edges(
    "validate",
    lambda state: "success" if not state.get("error_message") else "error",
    {
        "success": END,
        "error": END,
    }
)
validator_subgraph.set_entry_point("validate")

#### B. Subgrafo del Corrector
Este grafo es más simple. Su único propósito es corregir la query y devolverla al grafo principal, por lo que su nodo va directo a END.

In [58]:
corrector_subgraph = StateGraph(GraphState)
corrector_subgraph.add_node("correct", corrector_de_query_node)
corrector_subgraph.add_edge("correct", END)
corrector_subgraph.set_entry_point("correct")

#### C. Grafo Orquestador Principal
El orquestador une los subgrafos como si fueran nodos normales y define la lógica de bucle.

In [59]:
# Construcción del grafo principal (orquestador)
builder = StateGraph(GraphState)

# Añadimos los subgrafos como nodos al grafo principal
builder.add_node("validador_subgraph", validator_subgraph.compile())
builder.add_node("corrector_subgraph", corrector_subgraph.compile())
builder.add_node("finalizador", siguiente_tool_node)

# Definimos el flujo principal
builder.set_entry_point("validador_subgraph")

# Si el validador devuelve un estado con error, vamos al corrector. Si no, al finalizador.
builder.add_conditional_edges(
    "validador_subgraph",
    lambda state: "error" if state.get("error_message") else "success",
    {
        "success": "finalizador",
        "error": "corrector_subgraph",
    }
)

# Después de corregir, la query vuelve al validador para un nuevo intento.
builder.add_edge("corrector_subgraph", "validador_subgraph")

# El flujo termina después de la herramienta final.
builder.add_edge("finalizador", END)

# Compilamos el grafo
orchestrator = builder.compile()

### 4. Simulacion de casos del flujo de trabajo

#### A. query coherente sin fallo en la BBDD

In [60]:
user_msg_ok = "Quiero saber cuántos clientes hay en total."
query_ok = "SELECT COUNT(*) FROM adventure_works.dim_customer"
high_level_summary = "###Detalle de Tablas y Columnas Relevantes\n#### Tabla: dim_customer\n- *Descripción:* Contiene información detallada de clientes.\n- *Columnas Relevantes:\n  - customer_key: Clave primaria, tipo NUMERIC.\n  - customer_full_name: Nombre completo del cliente, tipo TEXT.\n\n#### Tabla: fact_sales\n- **Descripción:* Contiene detalles de las órdenes de ventas.\n- *Columnas Relevantes:*\n  - customer_key: Llave foránea, tipo NUMERIC.\n  - sales_amount: Subtotal de la línea de orden de venta, tipo NUMERIC.\n  - order_date: Fecha de creación del pedido, tipo DATE.\n"
inputs = {"user_message": user_msg_ok, "sql_query": query_ok, "high_level_summary": high_level_summary, "retries": 0}
for step in orchestrator.stream(inputs):
    print(step)


--- INICIANDO FASE DE JUEZ DE COHERENCIA ---
Veredicto del juez: COHERENTE

--- PASANDO A FASE DE EJECUCIÓN EN POSTGRESQL ---
{'validador_subgraph': {'user_message': 'Quiero saber cuántos clientes hay en total.', 'sql_query': 'SELECT COUNT(*) FROM adventure_works.dim_customer', 'high_level_summary': '###Detalle de Tablas y Columnas Relevantes\n#### Tabla: dim_customer\n- *Descripción:* Contiene información detallada de clientes.\n- *Columnas Relevantes:\n  - customer_key: Clave primaria, tipo NUMERIC.\n  - customer_full_name: Nombre completo del cliente, tipo TEXT.\n\n#### Tabla: fact_sales\n- **Descripción:* Contiene detalles de las órdenes de ventas.\n- *Columnas Relevantes:*\n  - customer_key: Llave foránea, tipo NUMERIC.\n  - sales_amount: Subtotal de la línea de orden de venta, tipo NUMERIC.\n  - order_date: Fecha de creación del pedido, tipo DATE.\n', 'db_output': {'schema': '\nCREATE TABLE dim_customer (\n\tcustomer_key SERIAL NOT NULL, \n\tgeography_key INTEGER, \n\tcustomer_a

In [39]:
#### B. Corrección por Incoherencia Semántica

In [61]:
# Escenario de incoherencia semántica: Query no relacionada con la pregunta
user_msg_ko_coherencia = "Quiero ver los nombres de los clientes.(solo 10)"
query_ko_coherencia = "SELECT SUM(sales_amount) FROM adventure_works.fact_sales" # Coherente sintácticamente, incoherente semánticamente
high_level_summary = "###Detalle de Tablas y Columnas Relevantes\n#### Tabla: dim_customer\n- *Descripción:* Contiene información detallada de clientes.\n- *Columnas Relevantes:\n  - customer_key: Clave primaria, tipo NUMERIC.\n  - first_name: Nombre del cliente, tipo TEXT.\n - last_name: Apellido del cliente, tipo TEXT.\n\n#### Tabla: fact_sales\n- **Descripción:* Contiene detalles de las órdenes de ventas.\n- *Columnas Relevantes:*\n  - customer_key: Llave foránea, tipo NUMERIC.\n  - sales_amount: Subtotal de la línea de orden de venta, tipo NUMERIC.\n  - order_date: Fecha de creación del pedido, tipo DATE.\n"

print("--- INICIANDO FLUJO PARA CORRECCIÓN DE INCOHERENCIA ---")
inputs = {"user_message": user_msg_ko_coherencia, "sql_query": query_ko_coherencia, "high_level_summary": high_level_summary, "retries": 0}

for step in orchestrator.stream(inputs):
    print(step)

--- INICIANDO FLUJO PARA CORRECCIÓN DE INCOHERENCIA ---

--- INICIANDO FASE DE JUEZ DE COHERENCIA ---
Veredicto del juez: INCOHERENTE

[X] La query es incoherente. No se ejecutará en la BBDD.
{'validador_subgraph': {'user_message': 'Quiero ver los nombres de los clientes.(solo 10)', 'sql_query': 'SELECT SUM(sales_amount) FROM adventure_works.fact_sales', 'high_level_summary': '###Detalle de Tablas y Columnas Relevantes\n#### Tabla: dim_customer\n- *Descripción:* Contiene información detallada de clientes.\n- *Columnas Relevantes:\n  - customer_key: Clave primaria, tipo NUMERIC.\n  - first_name: Nombre del cliente, tipo TEXT.\n - last_name: Apellido del cliente, tipo TEXT.\n\n#### Tabla: fact_sales\n- **Descripción:* Contiene detalles de las órdenes de ventas.\n- *Columnas Relevantes:*\n  - customer_key: Llave foránea, tipo NUMERIC.\n  - sales_amount: Subtotal de la línea de orden de venta, tipo NUMERIC.\n  - order_date: Fecha de creación del pedido, tipo DATE.\n', 'error_message': 'err

In [ ]:
#### C. Corrección de Fallo en la BBDD

In [69]:
# Escenario de error en BBDD: Tabla con nombre incorrecto
user_msg_ko_db = "Quiero saber el total de clientes por genero."
query_ko_db = "SELECT gender, COUNT(*) AS total_clients FROM adventure_works.dimen_customer GROUP BY gender" # Nombre de tabla incorrecto
high_level_summary = "###Detalle de Tablas y Columnas Relevantes\n#### Tabla: dim_customer\n- *Descripción:* Contiene información detallada de clientes.\n- *Columnas Relevantes:\n  - customer_key: Clave primaria, tipo NUMERIC.\n  - gender: genero del cliente, tipo TEXT.\n\n#### Tabla: fact_sales\n- **Descripción:* Contiene detalles de las órdenes de ventas.\n- *Columnas Relevantes:*\n  - customer_key: Llave foránea, tipo NUMERIC.\n  - sales_amount: Subtotal de la línea de orden de venta, tipo NUMERIC.\n  - order_date: Fecha de creación del pedido, tipo DATE.\n"

print("--- INICIANDO FLUJO PARA CORRECCIÓN DE ERROR EN BBDD ---")
inputs = {"user_message": user_msg_ko_db, "sql_query": query_ko_db, "high_level_summary": high_level_summary, "retries": 0}

for step in orchestrator.stream(inputs):
    print(step)

--- INICIANDO FLUJO PARA CORRECCIÓN DE ERROR EN BBDD ---

--- INICIANDO FASE DE JUEZ DE COHERENCIA ---
Veredicto del juez: COHERENTE

--- PASANDO A FASE DE EJECUCIÓN EN POSTGRESQL ---

[X] Error de PostgreSQL detectado: table_names {'dimen_customer'} not found in database
{'validador_subgraph': {'user_message': 'Quiero saber el total de clientes por genero.', 'sql_query': 'SELECT gender, COUNT(*) AS total_clients FROM adventure_works.dimen_customer GROUP BY gender', 'high_level_summary': '###Detalle de Tablas y Columnas Relevantes\n#### Tabla: dim_customer\n- *Descripción:* Contiene información detallada de clientes.\n- *Columnas Relevantes:\n  - customer_key: Clave primaria, tipo NUMERIC.\n  - gender: genero del cliente, tipo TEXT.\n\n#### Tabla: fact_sales\n- **Descripción:* Contiene detalles de las órdenes de ventas.\n- *Columnas Relevantes:*\n  - customer_key: Llave foránea, tipo NUMERIC.\n  - sales_amount: Subtotal de la línea de orden de venta, tipo NUMERIC.\n  - order_date: Fech